# Лабораторная работа 4.2

Вихляев Егор, ММТ-21

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## 0. Предварительный анализ данных

### 0.1. Загрузка данных

In [3]:
df = pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


#### Данные содержат следующие характеристики:

Pregnancies – число случаев беременности

Glucose – концентрация глюкозы в крови

BloodPressure – артериальное диастолическое давление (мм рт. ст.)

SkinThickness – толщина кожной складки трехглавой мышцы (мм)

Insulin – 2-х часовой сывороточный инсулин

BMI – индекс массы тела

DiabetesPedigreeFunction – числовой параметр наследственности диабета

Age – возраст

Outcome – целевая переменная: 1 – наличие заболевания, 0 – отсутствие


### 0.2. Распределение наблюдений

In [4]:
df.groupby('Outcome').value_counts()

Outcome  Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin  BMI   DiabetesPedigreeFunction  Age
0        0            57       60             0              0        21.7  0.735                     67     1
                      67       76             0              0        45.3  0.194                     46     1
         12           100      84             33             105      30.0  0.488                     46     1
         10           133      68             0              0        27.0  0.245                     36     1
                      139      80             0              0        27.1  1.441                     57     1
                                                                                                            ..
1        5            116      74             29             0        32.3  0.660                     35     1
                      124      74             0              0        34.0  0.220                     38     1
      

In [5]:
class_dist = df.groupby('Outcome').size()
print('Количество наблюдений в каждом классе:\n', class_dist)

Количество наблюдений в каждом классе:
 Outcome
0    500
1    268
dtype: int64


Как мы видим, наблюдения распределились неравномерно. Класс «0» явно превосходит класс «1» примерно в 1.8 раз. 

# 1. Подготовка данных 

### 1.1. Разделение данных на признаки и ответы, а затем на обучающую и тестовую выборки

In [6]:
X = df[['Pregnancies',  'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']] #Признаки
y = df['Outcome'] #Целевая переменная (ответы)

print('Объект X:')
print(X.head())

print('Объект y:')
print(y.head())

Объект X:
   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  
0                     0.627   50  
1                     0.351   31  
2                     0.672   32  
3                     0.167   21  
4                     2.288   33  
Объект y:
0    1
1    0
2    1
3    0
4    1
Name: Outcome, dtype: int64


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=101)
print('Размер обучающего набора данных (X_train):', X_train.shape)
print('Размер тестового набора данных (X_test):', X_test.shape)
print('Размер обучающего набора данных (y_train):', y_train.shape)
print('Размер тестового набора данных (y_test):', y_test.shape)

Размер обучающего набора данных (X_train): (691, 8)
Размер тестового набора данных (X_test): (77, 8)
Размер обучающего набора данных (y_train): (691,)
Размер тестового набора данных (y_test): (77,)


# 2. Обучение модели логистической регрессии

## 2.1. Обучение модели

### 2.1.1. Непосредственно обучаем модель

In [8]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)

/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

### 2.1.2. Выведем веса модели

In [9]:
coeffs = lr_model.coef_

print('Веса модели:')
for i, col in enumerate(X.columns):
    print(f"{col}: {round(coeffs[0][i],2)}")

Веса модели:
Pregnancies: 0.15
Glucose: 0.03
BloodPressure: -0.01
SkinThickness: -0.0
Insulin: -0.0
BMI: 0.07
DiabetesPedigreeFunction: 1.06
Age: 0.0


## 2.2. Оцениваем качество модели на тестовой выборке

In [10]:
from sklearn.metrics import classification_report

y_pred = lr_model.predict(X_test)

class_report = classification_report(y_test, y_pred)
print('Classification report:', class_report)

Classification report:               precision    recall  f1-score   support

           0       0.84      0.89      0.86        53
           1       0.71      0.62      0.67        24

    accuracy                           0.81        77
   macro avg       0.78      0.76      0.76        77
weighted avg       0.80      0.81      0.80        77



Начнем с того, что поскольку у нас сильный дисбаланс в классах, то $f1-score$ не рассчитывается, ибо предсказывается только наиболее распространенный класс. $f1$ же рассчитывается только для классов, которые присутствуют как в истинных метках, так и в предсказанных.

Точность (Precision):

1. Для класса 0 точность составляет 0.84, что означает, что 84% объектов, предсказанных как класс 0, действительно принадлежат к классу 0.
2. Для класса 1 точность составляет 0.71, что означает, что 71% объектов, предсказанных как класс 1, действительно принадлежат к классу 1.

Полнота (Recall):

1. Для класса 0 полнота составляет 0.89, что означает, что 89% объектов класса 0 было правильно обнаружено моделью.
2. Для класса 1 полнота составляет 0.62, что означает, что только 62% объектов класса 1 было обнаружено моделью.

Accuracy (точность): 

- 0.81 - доля правильно классифицированных объектов от общего числа объектов.

Macro avg (макро-усреднение): 

- среднее значение метрик для всех классов без учета их распределения в данных.

Weighted avg (взвешенное усреднение): 

- взвешенное среднее значение метрик для всех классов, пропорционально их поддержке в данных.

## 2.3. Обучим модель на стандартизированных данных

Нормализация данных признаковых переменных часто требуется в методах машинного обучения, особенно в алгоритмах, которые чувствительны к масштабу признаков, таких как метод ближайших соседей (k-Nearest Neighbors), метод опорных векторов (Support Vector Machines) или градиентный спуск.

Вот несколько случаев, когда нормализация данных может быть полезной:

1. Методы, основанные на расстоянии: Алгоритмы, которые используют расстояния между точками данных, такие как k-Nearest Neighbors, могут быть сильно повлияны на масштаб признаков. Если один из признаков имеет гораздо больший масштаб, чем другие, это может привести к тому, что расстояние будет определяться главным образом этим признаком.

2. Градиентный спуск: В алгоритмах оптимизации, таких как градиентный спуск, масштаб признаков может влиять на скорость сходимости. Нормализация признаков может помочь ускорить сходимость алгоритма.

3. Регуляризация: Некоторые алгоритмы машинного обучения, такие как регрессия Ridge или Lasso, могут быть чувствительны к масштабу признаков. Нормализация данных может помочь в справлении с этой проблемой.

Для модели логистической регрессии нормализация данных обычно не является обязательной, но может быть полезной в некоторых случаях.

Вот несколько соображений:

1. Масштабирование признаков: Логистическая регрессия не является алгоритмом, основанным на расстояниях, как, например, k-Nearest Neighbors. Однако масштабирование признаков может улучшить процесс обучения модели, ускорить сходимость и помочь в интерпретации коэффициентов.

2. Регуляризация: Если вы используете регуляризацию, такую как L1 (Lasso) или L2 (Ridge), масштабирование признаков может быть полезным. Это может помочь в том, чтобы штраф за коэффициенты признаков был более справедливым и однородным по всем признакам.

3. Интерпретация коэффициентов: Если вы хотите сравнивать важность признаков на основе их коэффициентов в модели, масштабирование может помочь сделать эти сравнения более справедливыми.

Таким образом, хотя нормализация данных не является обязательной для построения модели логистической регрессии, ее применение может быть полезным для улучшения производительности модели и упрощения интерпретации результатов.

In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [12]:
lr_model_scaled = LogisticRegression()
lr_model_scaled.fit(X_train_scaled, y_train)

LogisticRegression()

In [13]:
coeffs_scaled = lr_model_scaled.coef_

print('Веса модели:')
for i, col in enumerate(X.columns):
    print(f"{col}: {round(coeffs[0][i],2)}")

Веса модели:
Pregnancies: 0.15
Glucose: 0.03
BloodPressure: -0.01
SkinThickness: -0.0
Insulin: -0.0
BMI: 0.07
DiabetesPedigreeFunction: 1.06
Age: 0.0


In [14]:
y_pred_scaled = lr_model_scaled.predict(X_test_scaled)
class_report_scaled = classification_report(y_test, y_pred_scaled)
print('Classification report:', class_report_scaled)

Classification report:               precision    recall  f1-score   support

           0       0.84      0.91      0.87        53
           1       0.75      0.62      0.68        24

    accuracy                           0.82        77
   macro avg       0.80      0.77      0.78        77
weighted avg       0.81      0.82      0.81        77



Точность (Precision):

1. Для класса 0 точность составляет 0.84, что означает, что 84% объектов, предсказанных как класс 0, действительно принадлежат к классу 0. Это также, как и в предыдущем отчете.
2. Для класса 1 точность составляет 0.75, что также выше, чем в предыдущем отчете (0.71).

Полнота (Recall):

1. Для класса 0 полнота составляет 0.91, что выше, чем в предыдущем отчете (0.89).
2. Для класса 1 полнота составляет 0.62, что также также та же самая, как и в предыдущем отчете (0.62).

F1-мера (F1-score):

1. F1-мера для класса 0 составляет 0.87, что выше, чем в предыдущем отчете (0.86).
2. F1-мера для класса 1 составляет 0.68, что также выше, чем в предыдущем отчете (0.67).

Поддержка (Support):

1. Поддержка для каждого класса осталась той же самой.
2. Также в отчете представлены общие метрики для всех классов:

Accuracy (точность): 

- 0.82 - повышение точности на 1% по сравнению с предыдущими результатами.

Macro avg (макро-усреднение): 

- значение F1-меры немного повысилось до 0.78.

Weighted avg (взвешенное усреднение): 
- взвешенное среднее значение метрик также немного повысилось до 0.81.

##### Вывод: модель показывает небольшое улучшение!

## 2.4. Кросс-валидация

**Кросс-валидация (cross-validation)** - это метод оценки производительности модели машинного обучения, который позволяет оценить, насколько хорошо модель обобщает свои предсказательные способности на новых данных. Он представляет собой процедуру, при которой исходный набор данных разбивается на несколько подмножеств (называемых фолдами), после чего модель обучается на одном из фолдов и тестируется на оставшихся фолдах. Этот процесс повторяется несколько раз, и для каждого фолда вычисляется метрика производительности (например, точность, $F1$-мера или $ROC AUC$). Затем эти метрики усредняются для получения итоговой оценки качества модели.

In [15]:
from sklearn.model_selection import cross_val_score

scores_lr_model = cross_val_score(lr_model, X_train, y_train, cv=5)
scores_lr_model_scaled = cross_val_score(lr_model_scaled, X_train_scaled, y_train, cv=5)

/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_ite

**cv** — это количество фолдов.

Более высокое количество фолдов может привести к более надежной оценке производительности модели, поскольку каждый фолд будет содержать меньше данных, и модель будет оцениваться на большем числе различных подмножеств данных.

Меньшее количество фолдов может привести к более быстрой оценке производительности модели, особенно на больших наборах данных или для моделей, требующих длительного времени обучения.

Общее правило заключается в том, что нужно выбирать количество фолдов таким образом, чтобы балансировать между надежностью оценки и вычислительной нагрузкой. Обычно на практике используются значения от 3 до 10 для количества фолдов в перекрестной проверке.

In [16]:
print("Средняя точность модели lr_model без стандартизации на перекрестной проверке:", round(scores_lr_model.mean(),2))
print("Средняя точность модели lr_model_scaled с использованием стандартизации на перекрестной проверке:", round(scores_lr_model_scaled.mean(),2))

Средняя точность модели lr_model без стандартизации на перекрестной проверке: 0.75
Средняя точность модели lr_model_scaled с использованием стандартизации на перекрестной проверке: 0.77


##### Вывод: Средняя точность модели lr_model_scaled с использованием стандартизации (0.77) выше, чем у модели lr_model без стандартизации (0.75). Это указывает на то, что стандартизация признаков может улучшить производительность модели логистической регрессии.

## 2.5. Поиск по сетке

**Поиск по сетке (grid search)** - это метод поиска оптимальных гиперпараметров* для модели машинного обучения путем систематического перебора значений гиперпараметров из определенного набора. Для модели логистической регрессии гиперпараметры могут включать в себя коэффициент регуляризации (например, параметр C в логистической регрессии), тип регуляризации (L1 или L2) и другие параметры, которые могут влиять на процесс обучения модели.

Когда говорят о "качестве модели при нестандартизированных и для стандартизированных признаков почти равном", это может означать, что стандартизация признаков (например, масштабирование) не сильно влияет на результат модели. Однако, это не означает, что оптимальные значения гиперпараметров будут такими же для стандартизированных и нестандартизированных признаков.

Поэтому поиск по сетке может быть полезным для определения оптимальных гиперпараметров в обоих случаях. При этом вам необходимо будет определить набор значений гиперпараметров, которые вы хотите исследовать, а затем применить кросс-валидацию для каждой комбинации значений гиперпараметров, чтобы оценить их производительность на ваших данных. После этого вы можете выбрать комбинацию гиперпараметров, которая дает наилучшее качество модели.

В нашем случае, точности моделей при стандартизированных и нестандартизированных данных практически идентичны (0.75 и 0.77 соответственно). Это говорит о том, что модель логистической регрессии не чувствительна к масштабированию признаков. **В таком случае, поиск по сетке можно произвести для любой из этих моделей, поскольку результат будет примерно схожим. Но обычно рекомендуется использовать стандартизированные данные, поскольку это сделает процесс более стабильным и улучшит интепретируемость весов модели.** 

-----------

*: **Гиперпараметры** - это параметры, которые не учатся непосредственно во время обучения модели, а задаются вручную до начала процесса обучения и влияют на его ход и результат. Например, для модели логистической регрессии гиперпараметрами могут быть коэффициент регуляризации, тип регуляризации (L1 или L2), порог для принятия решения и так далее.

In [17]:
from sklearn.model_selection import GridSearchCV

#Определяем сетку гиперпарамметров
param_grid = {'C': [0.1, 1, 10], 'penalty': ['l2']}

#Создаем экземпляр класса GridSearchCV:
grid_search = GridSearchCV(lr_model_scaled, param_grid, cv=5, scoring='f1') #cv -- количество фолдов; scoring, очевидно, -- метрика

#Выполняем поиск по сетке
grid_search.fit(X_train_scaled, y_train)

#Получаем лучшие параметры и оценку
best_params = grid_search.best_params_
best_score = grid_search.best_score_

#Дополнительно оценим модель на тестовых данных
test_score = grid_search.score(X_test_scaled, y_test)

print('Лучшие параметры:', best_params)
print('Лучшая оценка:', round(best_score,2))

Лучшие параметры: {'C': 10, 'penalty': 'l2'}
Лучшая оценка: 0.63


Поскольку наши классы изначально несбалансированы, то было разумным в качестве scoring использовать именно $f1$-меру.

# 3. Метод опорных векторов

**Метод опорных векторов** — это мощный алгоритм машинного обучения, который используется как для классификации, так и для регрессии. Он строит гиперплоскости в пространстве признаков, которые лучше всего разделяют точки данных разных классов. Алгоритм работает над предположении, что чем больше расстояние (зазор) между разделяющей гиперплоскостью и объектами разделяемых классов, тем меньше будет средняя ошибка классификатора.

Гиперплоскость всегда имеет размерность на 1 меньше, чем пространство признаков. Например, для $\mathbb{R}^1$ гиперплоскостью является точка, для $\mathbb{R}^2$ — прямая, для $\mathbb{R}^3$ — плоскость и т.д. Гиперплоскость делит пространство на два полупространства. Говорят, что она разделяет два класса $C_1$ и $C_2$, если объекты этих классов лежат по разные стороны от гиперплоскости.

Сама гиперплоскость в пространстве $\mathbb{R}^n$ определяется уравнением $\langle \vec{w}, \vec{x} \rangle - b = 0$ при заданных $\vec{w}$ и $b$ — как множество векторов $\vec{x} = (x_1, \ ... \, x_n) \in \mathbb{R}^{n-1}$.

Ключевые преимущества SVM включают в себя:

1. Эффективность в пространствах высокой размерности.
2. Устойчивость к переобучению, особенно в случае небольшого количества обучающих данных.
3. Возможность использования различных ядерных функций для адаптации к нелинейным данным.

Однако, SVM также имеет недостатки, включая:

1. Сложность настройки гиперпараметров, таких как выбор ядра и параметра регуляризации.
2. Требование к масштабированию данных.
3. Относительно плохая производительность на больших наборах данных.

## 3.1. Обучим SVM-модель

In [25]:
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train, y_train)

SVC(kernel='linear', random_state=42)

## 3.2. Оценим качество SVM-модели на тестовой выборке

In [27]:
from sklearn.metrics import accuracy_score

y_pred_svm = svm_model.predict(X_test)
print('Classification report:\n', classification_report(y_test, y_pred_svm))

Classification report:
               precision    recall  f1-score   support

           0       0.82      0.89      0.85        53
           1       0.70      0.58      0.64        24

    accuracy                           0.79        77
   macro avg       0.76      0.74      0.75        77
weighted avg       0.79      0.79      0.79        77



Исходя из этого отчета, можно сказать, что модель хорошо справляется с классификацией примеров класса 0 (высокая точность и полнота), но у нее есть проблемы с классификацией примеров класса 1 (низкая полнота).

## 3.3. Построим SVM-модель на стандартизированных данных и оценим ее

In [28]:
svm_model_scaled = SVC(kernel='linear', random_state=42)
svm_model_scaled.fit(X_train_scaled, y_train)

y_pred_svm_scaled = svm_model_scaled.predict(X_test_scaled)
print('Classification report:\n', classification_report(y_test, y_pred_svm_scaled))

Classification report:
               precision    recall  f1-score   support

           0       0.83      0.91      0.86        53
           1       0.74      0.58      0.65        24

    accuracy                           0.81        77
   macro avg       0.78      0.74      0.76        77
weighted avg       0.80      0.81      0.80        77



Видим, что модель улучшилась во всех оценках, кроме macroavg/recall.

## 3.4. Оценим SVM-модель через кросс-валидацию

In [29]:
scores_svm_model = cross_val_score(svm_model, X_train, y_train, cv=5)
scores_svm_model_scaled = cross_val_score(svm_model_scaled, X_train_scaled, y_train, cv=5)

In [30]:
print("Средняя точность модели svm_model без стандартизации на перекрестной проверке:", round(scores_svm_model.mean(),2))
print("Средняя точность модели svm_model_scaled с использованием стандартизации на перекрестной проверке:", round(scores_svm_model_scaled.mean(),2))

Средняя точность модели svm_model без стандартизации на перекрестной проверке: 0.76
Средняя точность модели svm_model_scaled с использованием стандартизации на перекрестной проверке: 0.76


**Вывод:** SVM-модель показывает одинаковую среднюю точность при кросс-валидации как со стандартизированными, так и с нестандартизированными данными.

## 3.5. Поиск наилучшего значения kernel для SVM-модели

In [33]:
param_grid_svm = {'kernel': ['linear', 'rbf', 'poly', 'sigmoid']}
grid_search_svm = GridSearchCV(svm_model_scaled,param_grid_svm, cv=5)
grid_search_svm.fit(X_train_scaled, y_train)

print('Best parametres:', grid_search_svm.best_params_['kernel'])

Best parametres: linear


# 4. Метод kNN

Сперва кратко опишем суть метода собственным словами. Допустим, что нам необходимо провести классификацию объектов на два класса — красный или желтый. Нам дана некоторая обучающая выборка и целевой объект (серого цвета):

<img src='knn.png' width=900, heigth=240>

Необходимо определить, к какому классу относится серый объект.

Интуитивно мы видим, что поскольку все его соседи желтые, то и он сам должен быть желтым. Эта интуиция и отражает суть метода kNN — классифицировать целевой объект, основаваясь на том, к каким классам относятся объекты, максимально похожие на него.

**Вот основные шаги метода kNN:**

1. **Выбор числа соседей (k):** Это гиперпараметр, который определяет количество ближайших соседей, учитываемых при классификации или регрессии. Выбор значения k зависит от конкретной задачи и может влиять на качество модели.

2. **Вычисление расстояний:** Для каждого тестового объекта вычисляются расстояния до всех обучающих объектов в пространстве признаков. Расстояние обычно измеряется с использованием метрики, такой как евклидово расстояние, манхэттенское расстояние, косинусное расстояние и т. д.

3. **Собственно нахождение k ближайших соседей:** Выбираются k объектов обучающего набора, ближайших к тестовому объекту в соответствии с расстояниями.

4. **Определение класса (или значения) объекта:** Для задачи классификации, класс тестового объекта определяется путем голосования: объект относится к тому классу, который преобладает среди его k ближайших соседей. Для задачи регрессии, значение тестового объекта может быть определено как среднее (или взвешенное среднее) значений его k ближайших соседей.

5. **Оценка качества модели:** После классификации или регрессии всех тестовых объектов можно оценить качество модели с использованием метрик, таких как точность, F1-мера, среднеквадратическая ошибка и т. д.

In [23]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

Поскольку нам необходимо найти оптимальные значения k, которое является ничем иным, как гиперпараметром, то для этого нужно воспользовать поиском по сетке методом кросс-валидации.

Если бы оптимальное значение искать было не нужно, то и поиск по сетке мы бы не делали!

In [25]:
#Определяем оптимальные значения k
param_grid_knn = {'n_neighbors': range(1,11)} #Опробуем значения k от 1 до 10
knn_model = KNeighborsClassifier()
grid_search_knn = GridSearchCV(knn_model, param_grid_knn, cv=5)
grid_search_knn.fit(X_train_scaled, y_train)
best_k = grid_search_knn.best_params_['n_neighbors']

#Обучение модели kNN с оптимальным значением k
knn_model_optimal = KNeighborsClassifier(n_neighbors=best_k)
knn_model_optimal.fit(X_train_scaled, y_train)

#Оцениваем качество модели
y_pred_knn = knn_model_optimal.predict(X_test_scaled)
accuracy_knn = accuracy_score(y_test, y_pred_knn)

print('Отптимальное значение k:', best_k)
print('Accuracy kNN с оптимальным k:', round(accuracy_knn,2))

Отптимальное значение k: 9
Accuracy kNN с оптимальным k: 0.75


# 5. Дерево решений

Пусть задано некоторое бинарное дерево, в котором:

- каждой внутренней вершине $v$ приписан предикат $B_v: \mathbb{X} \rightarrow 0,1$;

- каждой листовой вершине $v$ припсано предсказание $c_v \in \mathbb{Y}$, где $\mathbb{Y}$ — область значений целевой переменной.

В ходе каждого предсказания происходит проход по этому дереву от корня к некоторому листу. В очередной внутренней вершине $v$ проход продолжится вправо, если $B_v(x) = 1$, или влево, если $B_v(x)=0$. Проход продолжится до момента, пока не будет достигнут некоторый лист. Ответом алгоритма на объекте $x$ считается предсказание $c_v$, приписанное этому листу.

В предикатах обычно используют сравнение с некоторым порогом $t \in \mathbb{R}$ по какому-то $j$-му признаку: $$B_v(x,j,t) = [x_j \leq t].$$

Из структуры дерева решений следуют определенный интресные свойства:

- выученная функция является кусочно-постоянной, а значит производная равна нулю на всей области определения. Таким образом, о градиентных методах можно даже и не думать;

- чтобы результат обладал хорошей обобщающей способностью (то есть мог что-то выучить), нам необходимо оставлять дерево как можно более простым;

- дерево решений (в отличие от линейной модели) не сможет экстраполировать зависимости за границы области значений обучающей выборки.

Так же следует отметить, что решаюшие деревья имеют низкую обобщающую способность, но поскольку они достаточно легки в построении, их часто используют как кирпичики в построении ансамблей — моделей, предсказания которых делаются на основе агрегации предсказаний других моделей.

Ниже на картинке слева представлен пример решающего дерева, а справа представлен пример соответствующей решаюшей поверхности, порождаемой дерево:

<img src='decTreeAndPlane.png' width=900, heigth=240>

In [30]:
from sklearn.tree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier()
dt_model.fit(X_train_scaled, y_train)

dt_accuracy = accuracy_score(y_test, dt_model.predict(X_test_scaled))
print('Точность (accuracy) модели дерева решений:', round(dt_accuracy,2))

Точность (accuracy) модели дерева решений: 0.69


# 6. Оценка качества дерева решений и kNN

In [34]:
y_pred_decTr = dt_model.predict(X_test_scaled)
print('- Classification Report для дерева решений:')
print(classification_report(y_test, y_pred_decTr))

y_pred_knn = knn_model_optimal.predict(X_test_scaled)
print('- Classification Report для kNN:')
print(classification_report(y_test, y_pred_knn))

- Classification Report для дерева решений:
              precision    recall  f1-score   support

           0       0.81      0.72      0.76        53
           1       0.50      0.62      0.56        24

    accuracy                           0.69        77
   macro avg       0.65      0.67      0.66        77
weighted avg       0.71      0.69      0.70        77

- Classification Report для kNN:
              precision    recall  f1-score   support

           0       0.79      0.87      0.83        53
           1       0.63      0.50      0.56        24

    accuracy                           0.75        77
   macro avg       0.71      0.68      0.69        77
weighted avg       0.74      0.75      0.74        77



# 7. Оптимальное значение гиперпараметра max_depth через поиск по сетке

Гиперпараметр max_depth (по-сути, максимальная глубина) является одним из важнейших параметров для регуляризации и контроля переобучения в построении деревьев решений и их ансамблей. В дереве решений, глубина дерева определяет, сколько раз дерево делит данные. Чем больше глубина — тем сложнее модель и тем больше вероятность переобучения. 

Гиперпараметр max-depth ограничивает максимальную глубину дерева, что позволяет контролировать его сложность и предотвращать переобучение модели.

Выбор оптимального значения max_depth зависит от конкретного набора данных и требует настройки при помощи кросс-валидации или других методов оценки моделей. 

In [45]:
param_grid_md = {'max_depth': [3, 5, 7, 9, None]}

grid_search_md = GridSearchCV(estimator=dt_model, param_grid=param_grid_md, cv=5)
grid_search_md.fit(X_train_scaled, y_train)

print('Оптимальное значение max_depth:', grid_search_md.best_params_['max_depth']) #Чтобы выводилось само значение, а не словарь, указываем это в квадратных скобках
print('Оптимальное значение качества модели:', round(grid_search.best_score_,2))

Оптимальное значение max_depth: 3
Оптимальное значение качества модели: 0.63


# 8. Обучим модель дерева решений с оптимальным max_depth

In [48]:
optimal_max_depth = grid_search_md.best_params_['max_depth']
optimal_dt_model = DecisionTreeClassifier(max_depth=optimal_max_depth)

optimal_dt_model.fit(X_train_scaled, y_train)

y_pred_opt_dt_model = optimal_dt_model.predict(X_test_scaled)
print(classification_report(y_test, y_pred_opt_dt_model))

              precision    recall  f1-score   support

           0       0.86      0.91      0.88        53
           1       0.76      0.67      0.71        24

    accuracy                           0.83        77
   macro avg       0.81      0.79      0.80        77
weighted avg       0.83      0.83      0.83        77



Здесь нужно сделать важное примечание. Чем отличается оценка, произведенная сразу после нахождения оптимального значения max_depth и новая, проведенная в этом разделе?

Первая оценка использует кросс-валидацию. Она оценивает модель на нескольких разбиениях обучающего набора анных, а затем усредняет показатели качества по всем фолдам. Такая оценка позволяет оценить обобшщающую способность модели и сделать выводы о ее эффективности на новых данных.

Вторая оценка производится уже на всем обучающем наборе данных с использованием найденного оптимального значения гиперпараметра. Она производится на отложенном тестовом наборе данных. Этот подход позволяет оценить производительность модели на реальных данных, которые она ранее не видела.

# 9. Сравним все полученные модели и сделаем выводы

В качестве метрик выберем следующие: accuracy, recall, f1, ROC-AUC. 

В совокупности мы имеем 7 моделей:
1. Модель логистической регрессии с нестандартизированными параметрами
2. Модель логистической регрессии со стандартизированными параметрами
3. SVM-модель с нестандартизированными параметрами
4. SVM-модель со стандартизированными параметрами
5. kNN
6. Дерево решений
7. Дерево решений с оптимальным max_depth

In [34]:
from sklearn.metrics import roc_auc_score

## 9.1. Оценка модели логистической регрессии с нестандартизированными данными

In [41]:
print('Classification report:\n', class_report)
y_proba = lr_model.predict_proba(X_test)[:, 1]
print('AUC-ROC:', round(roc_auc_score(y_test, y_proba),2))

Classification report:
               precision    recall  f1-score   support

           0       0.84      0.89      0.86        53
           1       0.71      0.62      0.67        24

    accuracy                           0.81        77
   macro avg       0.78      0.76      0.76        77
weighted avg       0.80      0.81      0.80        77

AUC-ROC: 0.89


Accuracy: 0.81

Recall (1): 0.62

f1-score (1): 0.67

## 9.2. Оценка модели логистической регрессии со стандартизированными данными

In [42]:
print('Classification report:\n', class_report_scaled)
y_proba_scaled = lr_model_scaled.predict_proba(X_test_scaled)[:, 1]
print('AUC-ROC:', round(roc_auc_score(y_test, y_proba_scaled),2))

Classification report:
               precision    recall  f1-score   support

           0       0.84      0.91      0.87        53
           1       0.75      0.62      0.68        24

    accuracy                           0.82        77
   macro avg       0.80      0.77      0.78        77
weighted avg       0.81      0.82      0.81        77

AUC-ROC: 0.9


Accuracy: 0.82

Recall (1): 0.62

f1-score (1): 0.68

# 9.3. Оценка SVM-модели с нестандартизированными данными

In [43]:
print('Classification report:\n', classification_report(y_test, y_pred_svm))
y_proba_svm = svm_model.predict_proba(X_test)[:, 1]
print('AUC-ROC:', round(roc_auc_score(y_test, y_proba_svm),2))

Classification report:
               precision    recall  f1-score   support

           0       0.82      0.89      0.85        53
           1       0.70      0.58      0.64        24

    accuracy                           0.79        77
   macro avg       0.76      0.74      0.75        77
weighted avg       0.79      0.79      0.79        77



AttributeError: predict_proba is not available when  probability=False